In [ ]:
import os
import requests

# API key and URL setup
MISTRAL_API_KEY = "9WIddRZCLm***AS6RWiVZkT5O******"   # API KEY
MISTRAL_CHAT_URL = "https://api.mistral.ai/v1/chat/completions"
DEFAULT_MODEL = "mistral-small-latest"   # Using a small, latest AI model

# System prompt - tells AI how to behave
SYSTEM_PROMPT = (
    """You are Kuber.AI, a friendly and unbiased AI financial assistant for India-based retail users. 
Your job is to act like a personal finance companion that is always available, clear, and judgment-free. 

Core Functions:
1. Personalized financial advice: Analyze user’s income, spending, and goals to give tailored, data-driven insights on budgeting, saving, and investments. 
2. Multilingual support: Be ready to answer in the user’s preferred language (e.g., Hindi, Hinglish, Kannada, or English).
3. Expense and income tracking: Help users understand their financial health by explaining income, expenses, debts, and investments in simple terms.
4. Goal-based planning: Assist users in planning and achieving short-term and long-term goals (e.g., buying a car, higher education, or travel savings).
5. Unbiased recommendations: Provide clear guidance without promoting or selling any specific financial products. Stay neutral and transparent.

Guidelines:
- Always keep answers concise, structured, and easy to read (use bullet points or short paragraphs).  
- Use simple, everyday language; avoid heavy financial jargon.  
- Always include this disclaimer at the end:  
  "This is general information, not investment advice."  
- If asked about purchases or investments, outline generic steps and remind the user to research further.  
- If the question is outside finance, politely decline and redirect the user to finance-related topics.

Tone:
- Friendly, approachable, and supportive  
- Neutral, unbiased, and trustworthy  
- Act like a companion who helps make money management simple and stress-free
"""
)

# List to keep chat history
chat_history = [
    {"role": "system", "content": SYSTEM_PROMPT} # First add system instructions
]

# Function to chat with AI
def chat_with_ai(user_input: str) -> str:
    chat_history.append({"role": "user", "content": user_input}) # Save user input in history
    
    headers = {
        "Authorization": f"Bearer {MISTRAL_API_KEY}",   # API key in header
        "Content-Type": "application/json",
    }
    payload = {
        "model": DEFAULT_MODEL,  # Which AI model to use
        "messages": chat_history,   # Send full chat history
        "temperature": 0.3        # Low temperature = safe & stable answers
    }
    
    r = requests.post(MISTRAL_CHAT_URL, headers=headers, json=payload, timeout=30)   # Make API call
    r.raise_for_status()      # Raise error if something goes wrong
    data = r.json()             # Get JSON response

    
    ai_reply = data["choices"][0]["message"]["content"]     # Get AI reply from response
    chat_history.append({"role": "assistant", "content": ai_reply})         # Save AI reply in history
    return ai_reply          # Return AI answer


# Infinite loop to take user input
while True:
    user_q = input("You: ")   # Get input from user
    if user_q.lower() in ["exit", "quit", "bye"]:   # Check if user wants to exit
        print("AI: Bye! 👋")  # Say goodbye
        break
    answer = chat_with_ai(user_q)   # Call function to get AI reply
    print("AI:", answer, "\n")     # Print AI's reply



 


AI: Silver is a precious metal that has been used for centuries in various forms, including jewelry, coins, and industrial applications. Here are some key points about silver:

- **Physical Properties**: Silver is a shiny, white metal that is malleable, ductile, and an excellent conductor of electricity and heat.
- **Uses**:
  - **Industrial**: Used in electronics, solar panels, and medical equipment.
  - **Jewelry and Decor**: Often used in making jewelry, silverware, and decorative items.
  - **Investment**: Silver bars, coins, and ETFs are popular investment options.
  - **Currency**: Historically used in coins and is still used in some countries today.
- **Market**: Silver prices fluctuate based on supply, demand, and economic conditions. It is often seen as a safer investment during economic uncertainty.

If you're interested in buying silver, here are some general steps:
1. **Research**: Understand the current market price and types of silver products available.
2. **Choose a Rep

**Gold and Silver Transaction Management System using FastAPI and MySQL**


In [11]:
import nest_asyncio
import uvicorn
from fastapi import FastAPI, Depends
from pydantic import BaseModel
from sqlalchemy import create_engine, Column, Integer, String, Float, DateTime
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, Session
from datetime import datetime

nest_asyncio.apply()  # for Jupyter Notebook

DB_USER = "root"
DB_PASSWORD = ""   # password hai 
DB_HOST = "127.0.0.1"
DB_NAME = "gold_silver_db"

DATABASE_URL = f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}/{DB_NAME}"

engine = create_engine(DATABASE_URL)
SessionLocal = sessionmaker(bind=engine, autocommit=False, autoflush=False)

Base = declarative_base()


class Transaction(Base):
    __tablename__ = "transactions"
    id = Column(Integer, primary_key=True, index=True)
    user_id = Column(Integer, index=True)
    metal_type = Column(String(20))   # "gold" or "silver"
    transaction_type = Column(String(20))  # "buy" or "sell"
    quantity = Column(Float)
    price_per_unit = Column(Float)
    total_amount = Column(Float)
    created_at = Column(DateTime, default=datetime.utcnow)


# create table if not exists
Base.metadata.create_all(bind=engine)


app = FastAPI()

# Dependency - DB Session
def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()



class TransactionCreate(BaseModel):
    user_id: int
    metal_type: str   # "gold" / "silver"
    transaction_type: str  # "buy" / "sell"
    quantity: float
    price_per_unit: float



@app.post("/add_transaction")
def add_transaction(data: TransactionCreate, db: Session = Depends(get_db)):
    total = data.quantity * data.price_per_unit
    new_txn = Transaction(
        user_id=data.user_id,
        metal_type=data.metal_type.lower(),
        transaction_type=data.transaction_type.lower(),
        quantity=data.quantity,
        price_per_unit=data.price_per_unit,
        total_amount=total
    )
    db.add(new_txn)
    db.commit()
    db.refresh(new_txn)
    return {"message": "Transaction added successfully!", "transaction_id": new_txn.id}


@app.get("/transactions")
def get_all_transactions(db: Session = Depends(get_db)):
    txns = db.query(Transaction).all()
    return txns


@app.get("/transactions/{user_id}")
def get_user_transactions(user_id: int, db: Session = Depends(get_db)):
    txns = db.query(Transaction).filter(Transaction.user_id == user_id).all()
    return txns


uvicorn.run(app, host="127.0.0.1", port=8000)


C:\Users\DELL\AppData\Local\Temp\ipykernel_4968\4031368317.py:22: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()
INFO:     Started server process [4968]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:64462 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:64461 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:64461 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:64461 - "GET /dos HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:64463 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:64463 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:64468 - "POST /add_transaction HTTP/1.1" 200 OK
INFO:     127.0.0.1:64472 - "GET /transactions HTTP/1.1" 200 OK
INFO:     127.0.0.1:64473 - "GET /transactions/6 HTTP/1.1" 200 OK
INFO:     127.0.0.1:64518 - "GET /transactions HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [4968]
